In [4]:
wraptostride = lambda wrap: {"stride": wrap, "filter_size": wrap*wrap}

since I can only do 2d conv, my 1d stream needs to be wrapped into a 2d one in order to convolve. Unfortunately, that needs to happen *at each layer*, because the previous layer will output a 1d stream again.

In [5]:
for x in range(0, 25):
    print x, "->", wraptostride(x)
    

0 -> {'stride': 0, 'filter_size': 0}
1 -> {'stride': 1, 'filter_size': 1}
2 -> {'stride': 2, 'filter_size': 4}
3 -> {'stride': 3, 'filter_size': 9}
4 -> {'stride': 4, 'filter_size': 16}
5 -> {'stride': 5, 'filter_size': 25}
6 -> {'stride': 6, 'filter_size': 36}
7 -> {'stride': 7, 'filter_size': 49}
8 -> {'stride': 8, 'filter_size': 64}
9 -> {'stride': 9, 'filter_size': 81}
10 -> {'stride': 10, 'filter_size': 100}
11 -> {'stride': 11, 'filter_size': 121}
12 -> {'stride': 12, 'filter_size': 144}
13 -> {'stride': 13, 'filter_size': 169}
14 -> {'stride': 14, 'filter_size': 196}
15 -> {'stride': 15, 'filter_size': 225}
16 -> {'stride': 16, 'filter_size': 256}
17 -> {'stride': 17, 'filter_size': 289}
18 -> {'stride': 18, 'filter_size': 324}
19 -> {'stride': 19, 'filter_size': 361}
20 -> {'stride': 20, 'filter_size': 400}
21 -> {'stride': 21, 'filter_size': 441}
22 -> {'stride': 22, 'filter_size': 484}
23 -> {'stride': 23, 'filter_size': 529}
24 -> {'stride': 24, 'filter_size': 576}


whoops, I lied. I can do 1d conv just fine.

In [6]:
import theano
from blocks.bricks.conv import Convolutional

note: it seems you can't import theano unless the very first way you try to import theano is "import theano". no fancy from imports or letting dependencies import it.

In [7]:
from audio2numpy import load, get_current_song

In [8]:
#data, samplerate = load(get_current_song())

In [9]:
#data

In [10]:
#%matplotlib inline

In [11]:
#from audio2numpy import display_song
#display_song(data)

In [12]:
import audiotags
nonlyrical = audiotags.getbytag("nonlyrical")
lyrical = audiotags.getbytag("lyrical")
semilyrical = audiotags.getbytag("semilyrical")
anylyrical = nonlyrical | lyrical | semilyrical

In [13]:
len(anylyrical)

614

In [14]:
len(semilyrical)

63

In [15]:
from audiotags import basedir

In [16]:
#get_current_song()

In [17]:
list(nonlyrical)[0]

u'audio/T12 - End of Line - Daft Punk.mp3'

In [1]:
from localdata import metadata

In [19]:
len(metadata)

614

In [20]:
{k: m["bitrate"] for k, m in metadata.items() if m["bitrate"] != 44100}

{}

In [21]:
#import audio2numpy
#reload(audio2numpy)

In [23]:
import os
song = 'dwhelper/Take On Me.mp3'
data, bitrate = load(os.path.join(basedir, song))
metadata[song] = {"len": len(data), "bitrate": bitrate, "seconds": len(data) / float(bitrate)}

loaded file in 0.565978050232


In [24]:
bitrate

44100

In [25]:
for song in (k for k, m in metadata.items() if m["bitrate"] != 44100):
    data, bitrate = load(os.path.join(basedir, song))
    metadata[song] = {"len": len(data), "bitrate": bitrate, "seconds": len(data) / float(bitrate)}

In [26]:
{k: m["bitrate"] for k, m in metadata.items() if m["bitrate"] != 44100}

{}

In [27]:
lengths = sorted([m["len"] for m in metadata.values()])

In [28]:
#import numpy
#from matplotlib import pyplot

In [29]:
#numpy.histogram(lengths)

In [30]:
#pyplot.hist([(l/44100.0)/60.0 for l in sorted(lengths)[5:-5]], 50)

In [31]:
9 * 9

81

and the number of ops that would take on a full song:

In [32]:
inputwidth = 280000
print lengths[0] / float(inputwidth)
9 * 9 * inputwidth/2

1.00975714286


11340000

10x conv 9 stride 2 -> 10x maxpool 2 stride 2 -> 30x conv 3 -> maxpool 3 -> ...

In [34]:
# This cell is how I tested things out to try to get the network param count down
import numpy
class L(object):
    def __init__(self, convcount, width, poolamount, stride=None):
        self.convcount = convcount
        self.width = width
        if stride is None:
            self.stride = max(poolamount / 3, 1)
        else:
            self.stride = stride
        self.poolamount = poolamount

    def outputs_size(self, input_width):
        applications_per_conv = input_width / self.stride
        ret = self.convcount * applications_per_conv / self.poolamount, self.convcount
        return ret
    
    def params_count(self, input_width, channel_count):
        parameters_per_conv = channel_count * self.width + 1
        z = self.convcount * parameters_per_conv
        return z
    
    def ops_count(self, input_width, channel_count):
        applications_per_conv = input_width / self.stride
        parameters_per_conv = channel_count * self.width
        return applications_per_conv * parameters_per_conv * self.convcount \
                + applications_per_conv * 2


class FC(object):
    def __init__(self, width):
        self.width = width

    def outputs_size(self, inputs_count):
        return self.width, 1

    def params_count(self, inputs_count, channel_count):
        return inputs_count * channel_count * self.width + self.width
    
    def ops_count(self, inputs_count, channel_count):
        return inputs_count * channel_count * self.width + self.width * 2


net = [
    L(10, 9, 2, 2),
    L(1, 1, 1, 1),
    L(10, 3, 3),
    L(1, 1, 1, 1),
    L(10, 5, 5),
    L(1, 1, 1, 1),
    L(10, 11, 11),
    L(1, 1, 1, 1),
    L(10, 21, 21),
    L(1, 1, 1, 1),
    L(10, 31, 31),
    L(1, 1, 1, 1),
    L(10, 41, 41),
    L(1, 1, 1, 1),
    L(10, 41, 41),
    L(1, 1, 1, 1),
    FC(128),
    FC(2)  # cut me off and replace me when respecializing
]
ops = 0
#layer_in = (int(numpy.median(lengths)), 2)
layer_in = (sorted(lengths)[5:-5][0], 2)
params = 0
import datetime
print "input:", layer_in
print "in seconds:", datetime.timedelta(seconds=layer_in[0] / 44100.0)
print "average length:", numpy.average(lengths)
print "in seconds:", datetime.timedelta(seconds=numpy.average(lengths)/44100.0)
import math
print "square image size that would be this big:", math.sqrt(layer_in[0])
for layer in net:
    ops += layer.ops_count(*layer_in)
    pc = layer.params_count(*layer_in)
    pc1 = layer.params_count(layer_in[0], 1)
    params += pc
    
    
    layer_in = layer.outputs_size(layer_in[0])
    print "out width:", layer_in, "params:", pc

import locale
locale.setlocale(locale.LC_ALL, 'en_US')
sflops = int(722.7e9)
fflops = int(5e12)

print "total ops to run the network once:", locale.format("%d", ops, grouping=True)
print "gpu peak flops:", fflops
print "(hopefully) network runs/second on slow gpu:", float(sflops)/float(ops)
print "(hopefully) network runs/second on fast gpu:", float(fflops)/float(ops)
print "param count:", locale.format("%d", params, grouping=True)

print len(net)

 input: (3739392, 2)
in seconds: 0:01:24.793469
average length: 11670584.2068
in seconds: 0:04:24.639098
square image size that would be this big: 1933.75075953
out width: (9348480, 10) params: 190
out width: (9348480, 1) params: 11
out width: (31161600, 10) params: 40
out width: (31161600, 1) params: 11
out width: (62323200, 10) params: 60
out width: (62323200, 1) params: 11
out width: (18885818, 10) params: 120
out width: (18885818, 1) params: 11
out width: (1284749, 10) params: 220
out width: (1284749, 1) params: 11
out width: (41443, 10) params: 320
out width: (41443, 1) params: 11
out width: (777, 10) params: 420
out width: (777, 1) params: 11
out width: (14, 10) params: 420
out width: (14, 1) params: 11
out width: (128, 1) params: 1920
out width: (2, 1) params: 258
total ops to run the network once: 6,676,519,040
gpu peak flops: 5000000000000
(hopefully) network runs/second on slow gpu: 108.245029434
(hopefully) network runs/second on fast gpu: 748.893243627
param count: 4,056
18

question: how do two convolutional layers typically combine? what would make sense? if you have 10 images, created by applying the first layer, does the second layer get applied to all of them? resulting in filters_a * filters_b images on the second layer? that sounds right, but then how do you get rid of the filtered images? -> answer is here: http://ufldl.stanford.edu/tutorial/supervised/ConvolutionalNeuralNetwork/

In [35]:
from blocks.bricks.conv import ConvolutionalLayer, ConvolutionalSequence, ConvolutionalActivation

In [36]:
input_size = sorted(lengths)[5:-5][0]
input_size

3739392

In [37]:
from blocks.bricks import Rectifier
convolutions = ConvolutionalSequence(
    layers=[
        ConvolutionalLayer(
            activation=Rectifier().apply,
            filter_size=(9, 1),
            num_filters=10,
            conv_step=(2, 1),
            pooling_size=(2, 1),
            pooling_step=(1, 1),),
        ConvolutionalActivation(
            activation=Rectifier().apply,
            filter_size=(1, 1),
            num_filters=1,),
        ConvolutionalLayer(
            activation=Rectifier().apply,
            filter_size=(3, 1),
            num_filters=10,
            conv_step=(1, 1),
            pooling_size=(3, 1),
            pooling_step=(1, 1),),
        ConvolutionalActivation(
            activation=Rectifier().apply,
            filter_size=(1, 1),
            num_filters=1,),
        ConvolutionalLayer(
            activation=Rectifier().apply,
            filter_size=(5, 1),
            num_filters=10,
            conv_step=(1, 1),
            pooling_size=(5, 1),
            pooling_step=(1, 1),),
        ConvolutionalActivation(
            activation=Rectifier().apply,
            filter_size=(1, 1),
            num_filters=1,),
        ConvolutionalLayer(
            activation=Rectifier().apply,
            filter_size=(11, 1),
            num_filters=10,
            conv_step=(4, 1),
            pooling_size=(11, 1),
            pooling_step=(1, 1),),
        ConvolutionalActivation(
            activation=Rectifier().apply,
            filter_size=(1, 1),
            num_filters=1,),
        ConvolutionalLayer(
            activation=Rectifier().apply,
            filter_size=(21, 1),
            num_filters=10,
            conv_step=(7, 1),
            pooling_size=(21, 1),
            pooling_step=(1, 1),),
        ConvolutionalActivation(
            activation=Rectifier().apply,
            filter_size=(1, 1),
            num_filters=1,),
        ConvolutionalLayer(
            activation=Rectifier().apply,
            filter_size=(31, 1),
            num_filters=10,
            conv_step=(10, 1),
            pooling_size=(31, 1),
            pooling_step=(1, 1),),
        ConvolutionalActivation(
            activation=Rectifier().apply,
            filter_size=(1, 1),
            num_filters=1,),
        ConvolutionalLayer(
            activation=Rectifier().apply,
            filter_size=(41, 1),
            num_filters=10,
            conv_step=(13, 1),
            pooling_size=(41, 1),
            pooling_step=(1, 1),),
        ConvolutionalActivation(
            activation=Rectifier().apply,
            filter_size=(1, 1),
            num_filters=1,),
        ConvolutionalLayer(
            activation=Rectifier().apply,
            filter_size=(41, 1),
            num_filters=10,
            conv_step=(13, 1),
            pooling_size=(41, 1),
            pooling_step=(1, 1),),
        ConvolutionalActivation(
            activation=Rectifier().apply,
            filter_size=(1, 1),
            num_filters=1,),
    ],
    image_size=(input_size, 1),
    num_channels=2)

In [40]:
convolutions.initialize()

AttributeError: 'NoneType' object has no attribute 'initialize'